In [1]:
import numpy as np
import scipy.io
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.signal import savgol_filter
from scipy.signal import find_peaks
from scipy.signal import butter, lfilter
from scipy.signal import butter, sosfilt, sosfreqz
from scipy.signal import welch
from scipy import signal
import os
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [3]:
#save_path = 'D:\EECE499\Features\GSRFeatures.xlsx'
save_path = '..\..\..\GSRFeatures.xlsx'

In [4]:
sample_rate = 128

In [5]:
def butter_bandpass(lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype='band', output='sos')
    return sos

def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos, data)
    return y

def butter_lowpass(cutoff, fs, order):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=6):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [8]:
data_frames = []

for user_id in range(1, 59):
    print('User ID: ' + str(user_id))
    
    mean_skin_resistance = []
    mean_first_derivative_res = []
    mean_abs_first_derivative_res = []
    mean_neg_first_derivative_res = []
    percentage_neg_first_derivative_res = []
    std_skin_resistance = []
    avg_minima_con = []
    #avg_rising_time_res =[]
    #log_power_density = []
    std_skin_conductance = []
    mean_first_derivative_con = []
    mean_abs_first_derivative_con = []
    mean_abs_second_derivative_con = []
    avg_minima_res = []
    #log_power_density
    psd_subband_01 = []
    psd_subband_02 = []
    psd_subband_03 = []
    psd_subband_04 = []
    zero_crossing_con_slow = []
    zero_crossing_con_very_slow = []
    psd_subband_11 = []
    psd_subband_12 = []
    psd_subband_13 = []
    psd_subband_14 = []
    psd_subband_15 = []
    psd_subband_16 = []
    psd_subband_17 = []
    psd_subband_18 = []
    psd_subband_19 = []
    psd_subband_10 = []
    
    presentation_id = []
    
    for clip_id in range(1, 37):
        #print('Clip ID: ' + str(clip_id))
        
        #D:\EECE499\Raw\MyECGFunc\ASCERTAIN_Raw
        #data_path = 'D:/EECE499/Raw/MyECGFunc/ASCERTAIN_Raw/GSRData/Movie_P' + str(user_id).zfill(2) + '/GSR_Clip' + str(clip_id) + '.mat'    
        data_path = './../../ASCERTAIN_Raw/GSRData/Movie_P' + str(user_id).zfill(2) + '/GSR_Clip' + str(clip_id) + '.mat'
        
        presentation_id.append(str(user_id).zfill(2) + str(clip_id).zfill(2))
        
        if os.path.isfile(data_path):
            
            signal = scipy.io.loadmat(data_path)
            resistance = signal['Data_GSR'][:,4]
            time_stamp = signal['Data_GSR'][:,0]
            #print(signal)
            
            x = signal['Data_GSR'][:,0]
            y = resistance
            #plt.plot(x,y)
            #plt.show()
            yhat = savgol_filter(y, 501, 3)
            #plt.plot(x, yhat)
            #plt.show()
            
            resistance = yhat
            conductance = 1 / yhat
            #print(np.mean(resistance))
            
            # mean skin resistance
            mean_skin_resistance.append(np.mean(resistance))
            #print(mean_skin_resistance)
            
            # mean of first derivative of skin resistance
            first_derivatives = np.diff(resistance) / np.diff(time_stamp)
            mean_first_derivative_res.append(np.mean(first_derivatives))
            # print(mean_first_derivative_res)
            
            # mean of the absolute values of the first derivatives of skin resistance
            mean_abs_first_derivative_res.append(np.mean(np.absolute(first_derivatives)))
            #print(mean_abs_first_derivative_res)
            
            # mean of the negative first derivates of skin resistance
            #print(len(first_derivatives[first_derivatives < 0]))
            #print(len(first_derivatives))
            mean_neg_first_derivative_res.append(np.mean(first_derivatives[first_derivatives < 0]))
            #print(mean_neg_first_derivative_res)
            
            # percentage of negative first derivative
            #((first_derivatives[first_derivatives < 0]).sum() / len(first_derivatives))
            percentage_neg_first_derivative_res.append(((first_derivatives[first_derivatives < 0]).sum() / len(first_derivatives)))
            
            # standard deviation of skin resistance
            std_skin_resistance.append(np.std(resistance))
            #print(std_skin_resistance)
            
            # average number of local minima in skin conductance
            peaks, _ = find_peaks(resistance)
            avg_minima_con.append(len(peaks) / len(resistance))
            #plt.plot(resistance)
            #plt.scatter(peaks, resistance[peaks], marker='x', color='red')
            #plt.show()
            #print(avg_minima_con)
            
            # log power density of 4 subbands
            #subband_1 = butter_lowpass_filter(resistance, 0.1, sample_rate, order=6)
            #subband_2 = butter_bandpass_filter(resistance, 0.1, 0.2, sample_rate, order=6)
            #subband_3 = butter_bandpass_filter(resistance, 0.2, 0.3, sample_rate, order=6)
            #subband_4 = butter_bandpass_filter(resistance, 0.3, 0.4, sample_rate, order=6)
            epsilon = np.finfo(float).eps
            if (sample_rate * 15 > len(resistance)):
                f = np.full(len(resistance), np.nan)
                P = np.full(len(resistance), np.nan)
            else:
                f, P = welch(resistance, sample_rate, nperseg = sample_rate * 15, noverlap = sample_rate * 10)
            #print(np.sum(P(f > 0 and f <= 0.1)))
            psd_subband_01.append(np.log(np.sum(P[(f > 0) & (f <= 0.1)]) + epsilon))
            psd_subband_02.append(np.log(np.sum(P[(f > 0.1) & (f <= 0.2)]) + epsilon))
            psd_subband_03.append(np.log(np.sum(P[(f > 0.2) & (f <= 0.3)]) + epsilon))
            psd_subband_04.append(np.log(np.sum(P[(f > 0.3) & (f <= 0.4)]) + epsilon))     
            #print(np.log(np.sum(P(f > 0.3 and f <= 0.4))))
            
            #plt.plot(time_stamp, subband_2)
            #plt.plot(time_stamp, subband_3, color='red')
            #plt.plot(time_stamp, subband_4, color='green')
            #plt.plot(time_stamp, subband_1, color='yellow')
            
            # standard deviation of skin conductance
            std_skin_conductance.append(np.std(conductance))
            #print(std_skin_conductance)
            
            # mean of first derivative of skin conductance
            first_derivatives = np.diff(conductance) / np.diff(time_stamp)
            mean_first_derivative_con.append(np.mean(first_derivatives))
            #print(mean_first_derivative_con)
            
            # mean of the absolute values of the first derivatives of skin conductance
            mean_abs_first_derivative_con.append(np.mean(np.absolute(first_derivatives)))
            
            # mean of the absolute values of the second derivative of skin conductance
            second_derivatives = np.diff(first_derivatives) / np.diff(time_stamp)[:-1]
            mean_abs_second_derivative_con.append(np.mean(np.absolute(second_derivatives)))
            #print(np.diff(time_stamp))
            
            # average number of local minima in skin resistance
            peaks, _ = find_peaks(conductance)
            avg_minima_res.append(len(peaks) / len(conductance))
            #print(conductance[peaks])
            #print(_)
            
            # log power density of 10 subbands
            psd_subband_11.append(np.log(np.sum(P[(f > 0.0) & (f <= 0.24)]) + epsilon))
            psd_subband_12.append(np.log(np.sum(P[(f > 0.24) & (f <= 0.48)]) + epsilon))
            psd_subband_13.append(np.log(np.sum(P[(f > 0.48) & (f <= 0.72)]) + epsilon))
            psd_subband_14.append(np.log(np.sum(P[(f > 0.72) & (f <= 0.96)]) + epsilon))
            psd_subband_15.append(np.log(np.sum(P[(f > 0.96) & (f <= 1.2)]) + epsilon))
            psd_subband_16.append(np.log(np.sum(P[(f > 1.2) & (f <= 1.44)]) + epsilon))
            psd_subband_17.append(np.log(np.sum(P[(f > 1.44) & (f <= 1.68)]) + epsilon))
            psd_subband_18.append(np.log(np.sum(P[(f > 1.68) & (f <= 1.92)]) + epsilon))
            psd_subband_19.append(np.log(np.sum(P[(f > 1.92) & (f <= 2.16)]) + epsilon))
            psd_subband_10.append(np.log(np.sum(P[(f > 2.16) & (f <= 2.4)]) + epsilon))
            
            # zero crossing rate of skin conductance slow response
            #slow_res = butter_lowpass(conductance, 0.2, sample_rate)
            #zero_crossing_con_slow.append(((slow_res[:-1] * slow_res[1:]) < 0).sum())
            
            # zero crossing rate of skin conductance very slow response
            #very_slow_res = butter_lowpass(conductance, 0.08, sample_rate)
            #zero_crossing_con_very_slow.append(((very_slow_res[:-1] * very_slow_res[1:]) < 0).sum())
            
    user_data = pd.DataFrame({
        'presentation_id' : presentation_id,
        'mean_skin_resistance' : mean_skin_resistance,
        'mean_first_derivative_res' : mean_first_derivative_res,
        'mean_abs_first_derivative_res' : mean_abs_first_derivative_res,
        'mean_neg_first_derivative_res' : mean_neg_first_derivative_res,
        'percentage_neg_first_derivative_res' : percentage_neg_first_derivative_res,
        'std_skin_resistance' : std_skin_resistance,
        'avg_minima_con' : avg_minima_con,
        'std_skin_conductance' : std_skin_conductance,
        'mean_first_derivative_con' : mean_first_derivative_con,
        'mean_abs_first_derivative_con' : mean_abs_first_derivative_con,
        'mean_abs_second_derivative_con' : mean_abs_second_derivative_con,
        'avg_minima_res' : avg_minima_res,
        'psd_subband_01' : psd_subband_01,
        'psd_subband_02' : psd_subband_02,
        'psd_subband_03' : psd_subband_03,
        'psd_subband_04' : psd_subband_04,
        'psd_subband_11' : psd_subband_11,
        'psd_subband_12' : psd_subband_12,
        'psd_subband_13' : psd_subband_13,
        'psd_subband_14' : psd_subband_14,
        'psd_subband_15' : psd_subband_15,
        'psd_subband_16' : psd_subband_16,
        'psd_subband_17' : psd_subband_17,
        'psd_subband_18' : psd_subband_18,
        'psd_subband_19' : psd_subband_19,
        'psd_subband_10' : psd_subband_10
    })
    
    #print(len(presentation_id))
    #print(len(mean_skin_resistance))
    #print(len(mean_first_derivative_res))
    #print(len(mean_abs_first_derivative_res))
    #print(len(mean_neg_first_derivative_res))
    #print(len(percentage_neg_first_derivative_res))
    #print(len(std_skin_resistance))
    #print(len(avg_minima_con))
    #print(len(std_skin_conductance))
    #print(len(mean_first_derivative_con))
    #print(len(mean_abs_first_derivative_con))
    #print(len(mean_abs_second_derivative_con))
    #print(len(avg_minima_res))
    #print(len(psd_subband_01))
    #print(len(psd_subband_02))
    #print(len(psd_subband_03))
    #print(len(psd_subband_04))
    #print(len(psd_subband_11))
    #print(len(psd_subband_12))
    #print(len(psd_subband_13))
    #print(len(psd_subband_14))
    #print(len(psd_subband_15))
    #print(len(psd_subband_16))
    #print(len(psd_subband_17))
    #print(len(psd_subband_18))
    #print(len(psd_subband_19))
    #print(len(psd_subband_10))
    
    data_frames.append(user_data)

User ID: 1
User ID: 2
User ID: 3
User ID: 4
User ID: 5
User ID: 6
User ID: 7


C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in greater
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in less_equal
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:116: RuntimeWarning: invalid value encountered in greater
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:116: RuntimeWarning: invalid value encountered in less_equal
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:117: RuntimeWarning: invalid value encountered in greater
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:117: RuntimeWarning: invalid value encountered in less_equal
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in greater
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in less_equal
C:\U

User ID: 8
User ID: 9


C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: RuntimeWarning: divide by zero encountered in true_divide
C:\Users\Danny\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: divide by zero encountered in true_divide


User ID: 10
User ID: 11
User ID: 12
User ID: 13
User ID: 14
User ID: 15
User ID: 16
User ID: 17
User ID: 18
User ID: 19
User ID: 20
User ID: 21
User ID: 22
User ID: 23
User ID: 24
User ID: 25
User ID: 26
User ID: 27
User ID: 28
User ID: 29
User ID: 30
User ID: 31
User ID: 32
User ID: 33
User ID: 34
User ID: 35
User ID: 36
User ID: 37
User ID: 38
User ID: 39
User ID: 40
User ID: 41
User ID: 42
User ID: 43
User ID: 44
User ID: 45
User ID: 46
User ID: 47
User ID: 48
User ID: 49
User ID: 50
User ID: 51
User ID: 52
User ID: 53
User ID: 54
User ID: 55
User ID: 56
User ID: 57
User ID: 58


In [9]:
GSR_Features = pd.concat(data_frames,ignore_index=True)

In [10]:
GSR_Features

,avg_minima_con,avg_minima_res,mean_abs_first_derivative_con,mean_abs_first_derivative_res,mean_abs_second_derivative_con,mean_first_derivative_con,mean_first_derivative_res,mean_neg_first_derivative_res,mean_skin_resistance,percentage_neg_first_derivative_res,...,psd_subband_12,psd_subband_13,psd_subband_14,psd_subband_15,psd_subband_16,psd_subband_17,psd_subband_18,psd_subband_19,std_skin_conductance,std_skin_resistance
0,0.008826,0.008826,5.198073e-12,0.006644,1.086936e-13,2.806951e-12,-0.003598,-0.006267,35742.376162,-0.005121,...,2.418040,-3.153890,-4.605442,-5.565476,-6.548193,-7.208471,-8.100074,-8.020904,7.715090e-08,98.818695
1,0.010868,0.010868,9.929272e-12,0.012424,1.051111e-13,-1.332249e-12,0.001672,-0.007073,35333.182716,-0.005376,...,4.476947,-2.257119,-3.741417,-5.045383,-6.425283,-6.812100,-7.648866,-7.629370,7.143445e-08,89.249290
2,0.008451,0.008319,1.473367e-11,0.019115,1.136581e-13,7.711063e-12,-0.009896,-0.016838,35797.161532,-0.014506,...,3.299578,-1.139468,-2.946844,-4.552895,-5.451923,-6.488533,-7.674897,-7.358155,1.781554e-07,229.857608
3,0.011120,0.011120,1.159695e-11,0.015179,1.149416e-13,1.796719e-12,-0.002336,-0.012313,36128.808740,-0.008757,...,6.170280,-0.517624,-2.954446,-4.210169,-6.247137,-6.787621,-8.126892,-7.924695,8.984379e-08,117.553668
4,0.014696,0.014696,1.284983e-11,0.016475,1.210268e-13,2.863116e-12,-0.003669,-0.014263,35760.537473,-0.010072,...,4.795046,-1.451628,-3.610787,-5.148072,-6.076997,-6.520453,-7.522291,-7.856154,9.517719e-08,122.095171
5,0.016836,0.016836,5.926993e-12,0.007683,1.164151e-13,2.412729e-12,-0.003126,-0.006753,36001.506973,-0.005404,...,3.537465,-2.330960,-4.597616,-5.612652,-6.523707,-6.851376,-7.594508,-7.632182,4.067841e-08,52.735971
6,0.008715,0.008715,5.389451e-12,0.006737,1.035384e-13,-9.723479e-13,0.001222,-0.003393,35314.476889,-0.002757,...,3.822226,-2.954671,-4.486823,-5.625707,-6.662440,-7.161722,-8.251459,-8.016891,4.959027e-08,61.899668
7,0.006334,0.006334,5.544282e-10,0.350921,7.813587e-13,5.021986e-10,-0.281023,-0.413646,35485.741621,-0.315972,...,6.924812,6.105067,5.430195,3.860235,2.999823,2.565862,1.667991,1.570853,3.479058e-06,2318.050124
8,0.021883,0.021988,6.107514e-12,0.007926,1.229227e-13,2.862313e-12,-0.003724,-0.007644,35990.421396,-0.005825,...,3.533659,-2.822996,-4.216385,-5.312436,-6.449915,-6.934914,-7.804858,-7.846809,6.382634e-08,82.828142
9,0.008252,0.008333,1.552442e-11,0.019605,1.099966e-13,-3.559741e-12,0.004473,-0.012585,35522.440422,-0.007566,...,5.342474,-1.327783,-3.645622,-4.991349,-5.930973,-6.641969,-7.484986,-7.989883,8.420285e-08,106.155371


In [11]:
GSR_Features.to_excel(save_path)

In [ ]:
user_id = 2

# features list
mean_skin_resistance = []
mean_first_derivative_res = []
mean_abs_first_derivative_res = []
mean_neg_first_derivative_res = []
#percentage_neg_first_derivative_res = []
std_skin_resistance = []
avg_minima_con = []
#avg_rising_time_res =[]
#log_power_density = []
std_skin_conductance = []
mean_first_derivative_con = []
mean_abs_first_derivative_con = []
mean_abs_second_derivative_con = []
avg_minima_res = []
#log_power_density

time_stamp

for clip_id in range(1, 37):
    # load the raw data
    data_path = './../ASCERTAIN_Raw/GSRData/Movie_P' + str(user_id).zfill(2) + '/GSR_Clip' + str(clip_id) + '.mat'
    signal = scipy.io.loadmat(data_path)
    resistance = signal['Data_GSR'][:,4]
    time_stamp = signal['Data_GSR'][:,0]

    # smoothing the curve
    #plt.plot(time_stamp, resistance)
    #plt.title('Signal before smoothing')
    #plt.show()
    
    resistance = savgol_filter(resistance, 101, 3)
    conductance = 1 / resistance
    
    #plt.plot(time_stamp, resistance)
    #plt.title('Signal after smoothing')
    #plt.show()

    # calculating the features
    mean_skin_resistance.append(np.mean(resistance))
    first_derivatives = np.diff(resistance) / np.diff(time_stamp)
    mean_first_derivative_res.append(np.mean(first_derivatives))
    mean_abs_first_derivative_res.append(np.mean(np.absolute(first_derivatives)))
    mean_neg_first_derivative_res.append(np.mean(first_derivatives[first_derivatives < 0]))
    std_skin_resistance.append(np.std(resistance))
    peaks, _ = find_peaks(resistance)
    avg_minima_con.append(len(peaks) / len(resistance))
    std_skin_conductance.append(np.std(conductance))
    first_derivatives = np.diff(conductance) / np.diff(time_stamp)
    mean_first_derivative_con.append(np.mean(first_derivatives))
    mean_abs_first_derivative_con.append(np.mean(np.absolute(first_derivatives)))
    second_derivatives = np.diff(first_derivatives) / np.diff(time_stamp)[:-1]
    mean_abs_second_derivative_con.append(np.mean(np.absolute(second_derivatives)))
    peaks, _ = find_peaks(conductance)
    #print(len(peaks) / len(conductance))
    #print(len(conductance))
    avg_minima_res.append(len(peaks) / len(conductance))
    #print(len(avg_minima_con))

# load the ASCERTAIN features for User ID and Clip ID
data_path = './../ASCERTAIN_Features/ASCERTAIN_Features/Dt_GSRFeatures.mat'
signal = scipy.io.loadmat(data_path)
#print(signal['GSRFeatures_58'][0,0][:,0])

plt.plot(np.array(mean_skin_resistance))
plt.plot(signal['GSRFeatures_58'][0, user_id - 1][:,0], color='red')
plt.title('Mean of Skin Resistance')
plt.show()
plt.plot(7*np.array(mean_first_derivative_res))
plt.plot(signal['GSRFeatures_58'][0, user_id - 1][:,1], color='red')
plt.title('Mean of First Derivatives of Resistance')
plt.show()
plt.plot(5*np.array(mean_abs_first_derivative_res))
plt.plot(signal['GSRFeatures_58'][0, user_id - 1][:,2], color='red')
plt.title('Mean of Absolute First Derivatives of Resistance')
plt.show()
plt.plot(5*np.array(mean_neg_first_derivative_res))
plt.plot(signal['GSRFeatures_58'][0, user_id - 1][:,3], color='red')
plt.title('Mean of Negative First Derivatives of Resistance')
plt.show()
plt.plot(np.array(std_skin_resistance))
plt.plot(signal['GSRFeatures_58'][0, user_id - 1][:,5], color='red')
plt.title('Standard Deviation of Resistance')
plt.show()
plt.plot(np.array(avg_minima_con))
plt.plot(signal['GSRFeatures_58'][0, user_id - 1][:,6], color='red')
plt.title('Average Number of Local Minima of Conductance')
plt.show()

In [ ]:
yhat = savgol_filter(y, 51, 3)
plt.plot(x, yhat)
plt.show()